# Exploration
#### Dedicated to purely exploring the data, observing trends, and seeing what quirks the dataset might uncover
For larger trends and analyses on the dataset, visit `results.ipynb`

In [1]:
from data.render import *
from data.load import *
from data.api import *

## Section 1: Critics vs General Public

The first dataset we're observing consists of 5000 entries of albums rated by users on a platform called AOTY (albumoftheyear.org). The highest rated album on this platform is 2015's "To Pimp a Butterfly" by Kendrick Lamar, with a rating of 95/100. *(1)*

In [2]:
show(get_rows(df_aoty, 0).to_frame().T)

rank,title,artist,release_date,genres,user_score,rating_count,album_link
1,To Pimp a Butterfly,Kendrick Lamar,"March 15, 2015","Conscious Hip Hop, Jazz Rap, West Coast Hip Hop",95,"28,594 ratings",https://www.albumoftheyear.org/album/29250-kendrick-lamar-to-pimp-a-butterfly.php


For curiosity's sake, let's examine the frequency of Kendrick Lamar entries within AOTY's top 5000 user-rated albums.

In [3]:
show(filter(df_aoty, 'artist', 'Kendrick Lamar'))
freq = count_occurrences(df_aoty, 'artist', 'Kendrick Lamar') / len(df_aoty) * 100
print(f'{str(freq)}% of albums by Kendrick Lamar exist in the T5000 rankings')


rank,title,artist,release_date,genres,user_score,rating_count,album_link
1,To Pimp a Butterfly,Kendrick Lamar,"March 15, 2015","Conscious Hip Hop, Jazz Rap, West Coast Hip Hop",95,"28,594 ratings",https://www.albumoftheyear.org/album/29250-kendrick-lamar-to-pimp-a-butterfly.php
2,"good kid, m.A.A.d city",Kendrick Lamar,"October 22, 2012","Conscious Hip Hop, West Coast Hip Hop",94,"22,283 ratings",https://www.albumoftheyear.org/album/3840-kendrick-lamar-good-kid-maad-city.php
399,Mr. Morale & The Big Steppers,Kendrick Lamar,"May 13, 2022","Conscious Hip Hop, West Coast Hip Hop",86,"19,284 ratings",https://www.albumoftheyear.org/album/494992-kendrick-lamar-mr-morale-the-big-steppers.php
694,DAMN.,Kendrick Lamar,"April 14, 2017","Conscious Hip Hop, West Coast Hip Hop",84,"20,892 ratings",https://www.albumoftheyear.org/album/76870-kendrick-lamar-damn.php
2092,Section.80,Kendrick Lamar,"July 2, 2011","Conscious Hip Hop, West Coast Hip Hop",81,"10,653 ratings",https://www.albumoftheyear.org/album/2725-kendrick-lamar-section80.php


0.1% of albums by Kendrick Lamar exist in the T5000 rankings


Although 0.1% sounds insignificant, every single one of Kendrick's studio albums makes the list, so by rater metrics on AOTY, he is a certainly a chart topper. It's even more impressive that two of his albums comprise the top 2 entries in the dataset.

This is where we introduce the other dataset, Spotify's Top 200. This dataset is taken directly from Spotify and contains the top 200 most streamed songs, of all time. Unsurprisingly capping out the list is "Blinding Lights" by The Weeknd, a song which dominated the Covid-19 & post-Covid streaming era:

In [4]:
show(get_rows(df_top100s, 0).to_frame().T)

title,artist,top genre,year,beats.per.minute,energy,danceability,loudness.dB,liveness,valance,length,acousticness,speechiness,popularity
Blinding Lights,The Weeknd,canadian contemporary r&b,2020,171,73,51,-6,9,33,200,0,6,91


But the reason I introduce the new dataset is I'm more interested in if AOTY's critic rankings match the general consensus of the public. That is, does "To Pimp a Butterfly" and "good kid m.A.A.d city's chart performance match its critic acclaim? Let's observe the frequency of Kendrick Lamar in the top 200:

In [5]:
show(filter(df_top100s, 'artist', 'Kendrick Lamar'))

title,artist,top genre,year,beats.per.minute,energy,danceability,loudness.dB,liveness,valance,length,acousticness,speechiness,popularity
HUMBLE.,Kendrick Lamar,conscious hip hop,2017,150,62,91,-7,10,42,177,0,10,81


Observe that Kendrick Lamar's best work (according to critics) doesn't make it anywhere on the Spotify Top 200 chart: instead, we see "HUMBLE" - a 2017 track from "DAMN" which was rated 694th on AOTY. *(1)*

This may suggest two different things. One possibility is that "HUMBLE" significantly outshines Kendrick Lamar's other work, but as a whole, "DAMN" is not that great of an album. Another possiblity is that the general public and the average critic do not agree on what makes good music. We need to dive deeper- we introduce a third and final component, Spotify's Web API: using their API endpoints for album metadata, we can find out if album popularity (according to Spotify metrics) align with critic rankings *(3)*

In [8]:
kendrick_albums = filter(df_aoty, 'artist', 'Kendrick Lamar').copy()
kendrick_albums.loc[:, 'popularity'] = kendrick_albums['title'].apply(lambda title: get_popularity(title))
kendrick_albums_sorted = kendrick_albums.sort_values(by='popularity', ascending=False).reset_index(drop=True)
kendrick_albums_sorted.insert(0, 'relative_rank', range(1, len(kendrick_albums_sorted) + 1))
show(kendrick_albums_sorted[['relative_rank', 'title', 'popularity']])

relative_rank,title,popularity
1,DAMN.,88
2,Mr. Morale & The Big Steppers,83
3,To Pimp a Butterfly,80
4,"good kid, m.A.A.d city",76
5,Section.80,74


To demonstrate the difference, here's a table of the relative rankings for user reviews on AOTY:

In [25]:
albums = get_col(filter(df_aoty, 'artist', 'Kendrick Lamar'), 'title').T.to_frame()
ratings = get_col(filter(df_aoty, 'artist', 'Kendrick Lamar'), 'user_score').T.to_frame()
albums.insert(0, 'rank', range(1, len(albums) + 1))
albums.insert(2, 'user_score', ratings)
show(albums)

rank,title,user_score
1,To Pimp a Butterfly,95
2,"good kid, m.A.A.d city",94
3,Mr. Morale & The Big Steppers,86
4,DAMN.,84
5,Section.80,81


The key discovery here is that critics (on AOTY) and the general public (Spotify users) do not necessarily agree on the ranking of albums. Popularity doesn't match the critic scores: in fact, besides "Section.80", the list almost appears flipped.